# Project - Sentiment Analysis Using Deep Learning - CNNs

<img src="banner.jpeg">

When it comes to text data, sentiment analysis is one of the most widely performed analysis on it. Sentiment Analysis has been through tremendous improvements from the days of classic methods to recent times where in the state of the art models utilize deep learning to improve the performance.

Convolutional Neural Networks or CNNs are the work-horse of the deep learning world. They have, in some sense, brought deep learning research into mainstream discussions. The advancements in the image classification world has left even humans behind.

<img src="cnn.png">
In this project, we will attempt at performing sentiment analysis utilizing the power of CNNs.

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

## Load Movie Review Dataset

In [3]:
dataset = pd.read_csv(r'../Unit 11 - Sentiment Analysis - Unsupervised Learning/movie_reviews.csv.bz2', compression='bz2')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


In [4]:
# take a peek at the data
print(dataset.head())
reviews = np.array(dataset['review'])
sentiments = np.array(dataset['sentiment'])

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


### Prepare Train Test Split

In [5]:
# build train and test datasets
train_reviews = reviews[:35000]
train_sentiments = sentiments[:35000]
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]

## Preprocessing

To prepare text data for our deep learning model, we transform each review into a sequence.
Every word in the review is mapped to an integer index and thus the sentence turns into a sequence of numbers.

To perform this transformation, keras provides the ```Tokenizer```

In [6]:
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(train_reviews)

In [7]:
train_sequences = t.texts_to_sequences(train_reviews)

In [8]:
test_sequences = t.texts_to_sequences(test_reviews)

In [9]:
print("Vocabulary size={}".format(len(t.word_counts)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=105664
Number of Documents=35000


### Sequence Normalization

Not all reviews are of same length. To handle this difference in length of reviews, we define a maximum length.
For reviews which are smaller than this length, we padd them with zeros which longer ones are truncated

In [10]:
MAX_SEQUENCE_LENGTH = 500

In [11]:
# pad dataset to a maximum review length in words
X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Encoding Labels

The dataset contains labels of the form positive/negative. The following step encodes the labels using ```sklearn's``` ```LabelEncoder```

In [12]:
le = LabelEncoder()
num_classes=2 # positive -> 1, negative -> 0

In [13]:
y_train = le.fit_transform(train_sentiments)
y_test = le.transform(test_sentiments)

In [14]:
VOCAB_SIZE = len(t.word_counts)+1

## Prepare the Model

Since textual data is a sequence of words, we utilize ```1D``` convolutions to scan through the sentences.
The model first transforms each word into lower dimensional embedding/vector space followed by 1d convolutions and then passing the data through dense layers before the final layer for classification

In [18]:
EMBED_SIZE = 32
EPOCHS=5
BATCH_SIZE=128

In [19]:
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           3381280   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               2000250   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 251       
Total params: 5,384,885
Trainable params: 5,384,885
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [20]:
# Fit the model
model.fit(X_train, y_train, 
          validation_split=0.2,
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          verbose=1)

Train on 28000 samples, validate on 7000 samples
Epoch 1/5
28000/28000 [==============================] - 458s 16ms/step - loss: 0.3918 - acc: 0.8046 - val_loss: 0.2607 - val_acc: 0.8926
Epoch 2/5
28000/28000 [==============================] - 450s 16ms/step - loss: 0.1218 - acc: 0.9562 - val_loss: 0.2909 - val_acc: 0.8896
Epoch 3/5
28000/28000 [==============================] - 448s 16ms/step - loss: 0.0330 - acc: 0.9909 - val_loss: 0.3640 - val_acc: 0.8914
Epoch 4/5
28000/28000 [==============================] - 463s 17ms/step - loss: 0.0081 - acc: 0.9986 - val_loss: 0.4534 - val_acc: 0.8886
Epoch 5/5
28000/28000 [==============================] - 457s 16ms/step - loss: 0.0014 - acc: 0.9999 - val_loss: 0.5190 - val_acc: 0.8879


## Model Evaluation

In [21]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 89.23%


In [28]:
predictions = model.predict_classes(X_test)
predictions[:10]

array([[0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1]])

In [29]:
predictions = [item[0] for item in predictions]
predictions[:10]

[0, 1, 0, 1, 1, 0, 1, 0, 1, 1]

In [30]:
predictions = ['positive' if item == 1 else 'negative' for item in predictions]
predictions[:10]

['negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive']

In [31]:
import model_evaluation_utils as meu

meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=predictions, 
                                      classes=['positive', 'negative']) 

Model Performance metrics:
------------------------------
Accuracy: 0.8923
Precision: 0.8925
Recall: 0.8923
F1 Score: 0.8923

Model Classification report:
------------------------------
              precision    recall  f1-score   support

    positive       0.90      0.88      0.89      7510
    negative       0.88      0.91      0.89      7490

   micro avg       0.89      0.89      0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000


Prediction Confusion Matrix:
------------------------------
                 Predicted:         
                   positive negative
Actual: positive       6604      906
        negative        710     6780
